# Chains

Chains allow us to combine multiple components together to create a single, coherent application. For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM. We can build more complex chains by combining multiple chains together, or by combining chains with other components.

## LLMChain

An `LLMChain` is a simple chain that adds some functionality around language models. It consists of a `PromptTemplate` and a language model (either an `LLM` or `ChatModel`). It formats the prompt template using the input key values provided, passes the formatted string to LLM and returns the LLM output.

Here's an example with single input:

In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

prompt = PromptTemplate(
    template="What is a good name for a company that makes {product}?",
    input_variables=["product"],
)

llm = OpenAI()
llm_chain = LLMChain(llm=llm, prompt=prompt)
llm_chain.predict(product="cars")

'\n\nAutoMotive Dynamics.'

To see what is happening under the hood, we can use the `verbose` parameter in `LLMCHain`:

In [2]:
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=True)
llm_chain.predict(product="cars")



> Entering new LLMChain chain...
Prompt after formatting:
What is a good name for a company that makes cars?

> Finished chain.


'\n\nAutobahn Motors.'

Here's an example with multiple inputs:

In [3]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about {subject}.",
    input_variables=["adjective", "subject"],
)

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
llm_chain.predict(adjective="sad", subject="ducks")



> Entering new LLMChain chain...
Prompt after formatting:
Tell me a sad joke about ducks.

> Finished chain.


'\n\nQ: What do you call a sad duck?\nA: A weep-quack.'

## Sequential Chain

The next step after calling a language model is to make a series of calls to a language model. This is particularly useful when you want to take the output from one call and use it as the input to another. `SequentialChain` allow you to connect multiple chains and compose them into pipelines that execute some specific scenario.

In [7]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.

llm = OpenAI(temperature=0.7)
synopsis_template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""

synopsis_prompt_template = PromptTemplate(
    template=synopsis_template,
    input_variables=["title", "era"],
)

synopsis_chain = LLMChain(
    llm=llm,
    prompt=synopsis_prompt_template,
    output_key="synopsis",
    verbose=True,
)

In [8]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=0.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["synopsis"],
)

review_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    output_key="review",
    verbose=True,
)

In [9]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True,
)


response = overall_chain(
    {"title": "Tragedy at sunset on the beach", "era": "Victorian England"}
)
response



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: Tragedy at sunset on the beach
Era: Victorian England
Playwright: This is a synopsis for the above play:

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:


Set in Victorian England, Tragedy at Sunset on the Beach is a drama that explores the consequences of a forbidden romance between two members of the upper class. Jessica and Edward are two young lovers from wealthy families, who have been secretly meeting on the beach at sunset every night. They know that their families would never approve of their union, but their love for each other is too strong to ignore.

One night, their clandestine

{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': "\n\nSet in Victorian England, Tragedy at Sunset on the Beach is a drama that explores the consequences of a forbidden romance between two members of the upper class. Jessica and Edward are two young lovers from wealthy families, who have been secretly meeting on the beach at sunset every night. They know that their families would never approve of their union, but their love for each other is too strong to ignore.\n\nOne night, their clandestine meetings on the beach are discovered by Jessica's brother, who threatens to tell their families. In a desperate attempt to keep their secret, Edward challenges Jessica's brother to a duel. In the ensuing fight, Edward is mortally wounded and dies in Jessica's arms.\n\nAs the sun sets on the beach, Jessica is overcome with grief and guilt. Her beloved Edward is gone, and she must now face the consequences of her actions. Tragedy at Sunset on the Beach is a stor

You can access the each output using their key:

In [12]:
print(response["synopsis"])



Set in Victorian England, Tragedy at Sunset on the Beach is a drama that explores the consequences of a forbidden romance between two members of the upper class. Jessica and Edward are two young lovers from wealthy families, who have been secretly meeting on the beach at sunset every night. They know that their families would never approve of their union, but their love for each other is too strong to ignore.

One night, their clandestine meetings on the beach are discovered by Jessica's brother, who threatens to tell their families. In a desperate attempt to keep their secret, Edward challenges Jessica's brother to a duel. In the ensuing fight, Edward is mortally wounded and dies in Jessica's arms.

As the sun sets on the beach, Jessica is overcome with grief and guilt. Her beloved Edward is gone, and she must now face the consequences of her actions. Tragedy at Sunset on the Beach is a story of forbidden love and its consequences, as well as the power of redemption.


In [13]:
print(response["review"])



Tragedy at Sunset on the Beach is a compelling and powerful drama that explores the consequences of a forbidden romance between two members of the upper class. The story follows Jessica and Edward, two young lovers whose love for each other is too strong to ignore. Despite the danger, they continue to meet on the beach at sunset every night. When their secret is discovered, their love is tested as Edward challenges Jessica's brother to a duel and is mortally wounded.

The play is beautifully written and the actors do a wonderful job of bringing the characters to life. The tension between Jessica and Edward is palpable, and their love for each other is both heartbreaking and inspiring. The set and costumes are also magnificent, transporting the audience to Victorian England.

Tragedy at Sunset on the Beach is a powerful and heartbreaking story of forbidden love and its consequences, as well as the power of redemption. It is a must-see for anyone looking for an emotionally moving theat

**TODO**: Write a introductory section for LangChain Expression Language.